# Test how to plot CPMG with widgets
Let us try from this

* [Using Interact](http://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)
* [Widget List](http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)


In [61]:
from IPython.display import HTML
HTML('<iframe width="213" height="120" src="https://www.youtube.com/embed/p7Hr54VhOp0" allowfullscreen></iframe>')

# Interactive plotting

In [62]:
# Import python packages
import numpy as np
# Plotting
import matplotlib.pylab as plt
%matplotlib inline

# Import relax modules
import os, sys, pathlib
sys.path.append( os.path.join(str(pathlib.Path.home()), "software", "relax" ))

# Import relax target function that prepare data
from target_functions import relax_disp
# Import library functions for each model
from lib.dispersion import cr72
from lib.dispersion import ns_cpmg_2site_expanded

## 'CR72'

In [63]:
# Setup parameters
def cr72_calc(
            w0_1H_s1=750., w0_1H_s2=750., 
            R20_s1=13.9, R20_s2=13.9, 
            dw_s1=1.02, dw_s2=0.69,
            pA_s1=0.87, pA_s2=0.5, 
            kex_s1=4027., kex_s2=4061.):
    """
    @keyword cpmg_e:  The end value of the CPMG pulse train. In Hz.
    @keyword isotope: The isotope of nuclei. Either 1H, 15N or 13C.
    @keyword w0_1H:   The spin Larmor frequencies for proton. In MHz.
    @keyword R20:     The transversal relaxation rate. In rad/s.
    @keyword dw:      The chemical shift difference between states A and B (in ppm).
    @keyword pA:      The population of state A.
    @keyword kex:     The exchange rate. In rad/s
    """
    cpmg_e=2500
    isotope='15N'
    # Gyromagnetic Ratio in [MHz/T]
    # http://bio.groups.et.byu.net/LarmourFreqCal.phtml
    g = {'1H':42.576, '15N':4.3156, '13C':10.705}
    # Magnet Field Strength [T]
    B0_s1 = w0_1H_s1 / g['1H']
    B0_s2 = w0_1H_s2 / g['1H']
    # Larmor frequency for isotope [MHz]
    w0_isotope_s1 = g[isotope]*B0_s1
    w0_isotope_s2 = g[isotope]*B0_s2

    # For simpel model, R20A and R20B is the same
    R20A_s1 = R20B_s1 = R20_s1
    R20A_s2 = R20B_s2 = R20_s2
    # Convert dw in ppm to rad/s
    dw_rad_s1 = dw_s1 * w0_isotope_s1*2*np.pi
    dw_rad_s2 = dw_s2 * w0_isotope_s2*2*np.pi
    
    # Make x values. In Hz.
    x_cpmg_frqs = np.linspace(10, cpmg_e, num=100)
    
    # Make empty y_val
    y_R2_s1 = np.zeros(x_cpmg_frqs.size)
    y_R2_s2 = np.zeros(x_cpmg_frqs.size)
    # Calculate y, and make in-memore replacement in y
    cr72.r2eff_CR72(r20a=R20A_s1, r20a_orig=R20A_s1, r20b=R20B_s1, r20b_orig=R20B_s1, 
           pA=pA_s1, dw=dw_rad_s1, dw_orig=dw_rad_s1, kex=kex_s1, 
           cpmg_frqs=x_cpmg_frqs, back_calc=y_R2_s1)
    # For spin 2
    cr72.r2eff_CR72(r20a=R20A_s2, r20a_orig=R20A_s2, r20b=R20B_s2, r20b_orig=R20B_s2, 
           pA=pA_s2, dw=dw_rad_s2, dw_orig=dw_rad_s2, kex=kex_s2, 
           cpmg_frqs=x_cpmg_frqs, back_calc=y_R2_s2)
    
    # Make figure
    f, ax = plt.subplots(1, figsize=(12, 4))
    # Plot
    label_s1 = "sfrq=%.1f MHz\nR20=%.1f rad/s\ndw=%.1f ppm\npA=%.3f \nkex=%.1f rad/s"%(w0_1H_s1, R20_s1, dw_s1, pA_s1, kex_s1)
    plt.plot(x_cpmg_frqs, y_R2_s1, label=label_s1)
    label_s2 = "sfrq=%.1f MHz\nR20=%.1f rad/s\ndw=%.1f ppm\npA=%.3f \nkex=%.1f rad/s"%(w0_1H_s2, R20_s2, dw_s2, pA_s2, kex_s2)
    plt.plot(x_cpmg_frqs, y_R2_s2, label=label_s2)
    # Set labels
    plt.xlabel = "CPMG pulse train frequency v [Hz]"
    plt.ylabel = "R2,eff rad/s"
    ax = plt.gca()
    p_ylim_up = ax.get_ylim()[-1]
    # Round up to nearest 5
    p_ylim_up = p_ylim_up + (- p_ylim_up % 5 )
    ax.set_ylim(0, p_ylim_up)
    ax.set_xlim(0, cpmg_e)
    # Put legend outside
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    # Final call to show.
    plt.show()

# Widgets
from ipywidgets import interact, interactive
import ipywidgets as widgets

widget = interactive(cr72_calc, 
                       w0_1H_s1=(500., 1000., 50), w0_1H_s2=(500., 1000., 50),
                       R20_s1=(5.0, 25.0, 1.), R20_s2=(5.0, 25.0, 1.),
                       dw_s1=(0.1, 10., 0.1), dw_s2=(0.1, 10., 0.1),
                       pA_s1=(0.500, 1.000, 0.01), pA_s2=(0.500, 1.000, 0.01),
                       kex_s1=(400., 10000, 200.), kex_s2=(400., 10000, 200.))

#output = widget.children[-1]
#output.layout.height = '500px'
widget

interactive(children=(FloatSlider(value=750.0, description='w0_1H_s1', max=1000.0, min=500.0, step=50.0), FloatSlider(value=750.0, description='w0_1H_s2', max=1000.0, min=500.0, step=50.0), FloatSlider(value=13.9, description='R20_s1', max=25.0, min=5.0, step=1.0), FloatSlider(value=13.9, description='R20_s2', max=25.0, min=5.0, step=1.0), FloatSlider(value=1.02, description='dw_s1', max=10.0, min=0.1), FloatSlider(value=0.69, description='dw_s2', max=10.0, min=0.1), FloatSlider(value=0.87, description='pA_s1', max=1.0, min=0.5, step=0.01), FloatSlider(value=0.5, description='pA_s2', max=1.0, min=0.5, step=0.01), FloatSlider(value=4027.0, description='kex_s1', max=10000.0, min=400.0, step=200.0), FloatSlider(value=4061.0, description='kex_s2', max=10000.0, min=400.0, step=200.0), Output()), _dom_classes=('widget-interact',))

## 'NS CPMG 2-site expanded'

In [64]:
# Setup parameters
def ns_cpmg_2site_expanded_calc(relax_time=0.06,
            w0_1H_s1=750., w0_1H_s2=750., 
            R20_s1=13.9, R20_s2=13.9, 
            dw_s1=1.02, dw_s2=0.69,
            pA_s1=0.87, pA_s2=0.5, 
            kex_s1=4027., kex_s2=4061.):
    """
    @keyword cpmg_e:     The end value of the CPMG pulse train. In Hz.
    @keyword isotope:    The isotope of nuclei. Either 1H, 15N or 13C.
    @keyword relax_time: The experiment specific fixed time period for relaxation (in seconds).
    @keyword w0_1H:      The spin Larmor frequencies for proton. In MHz.
    @keyword R20:        The transversal relaxation rate. In rad/s.
    @keyword dw:         The chemical shift difference between states A and B (in ppm).
    @keyword pA:         The population of state A.
    @keyword kex:        The exchange rate. In rad/s
    """
    cpmg_e=2500
    isotope='15N'
    # Gyromagnetic Ratio in [MHz/T]
    # http://bio.groups.et.byu.net/LarmourFreqCal.phtml
    g = {'1H':42.576, '15N':4.3156, '13C':10.705}
    # Magnet Field Strength [T]
    B0_s1 = w0_1H_s1 / g['1H']
    B0_s2 = w0_1H_s2 / g['1H']
    # Larmor frequency for isotope [MHz]
    w0_isotope_s1 = g[isotope]*B0_s1
    w0_isotope_s2 = g[isotope]*B0_s2

    # Convert dw in ppm to rad/s
    dw_rad_s1 = dw_s1 * w0_isotope_s1*2*np.pi
    dw_rad_s2 = dw_s2 * w0_isotope_s2*2*np.pi
    
    # Make x values. In Hz.
    x_cpmg_frqs = np.linspace(40, cpmg_e, num=100)
    
    # Make empty y_val
    y_R2_s1 = np.zeros(x_cpmg_frqs.size)
    y_R2_s2 = np.zeros(x_cpmg_frqs.size)

    # Calculate y, and make in-memore replacement in y
    inv_relax_time = 1.0 / relax_time
    # Collect power
    power_arr = []
    tau_cpmg_arr = []
    for cpmg_frq in x_cpmg_frqs:
        # num_cpmg
        power = int(round(cpmg_frq * relax_time))
        power_arr.append(power)
        # tcp
        tau_cpmg = 0.25 * relax_time / power
        tau_cpmg_arr.append(tau_cpmg)
    # Conver to numpy
    num_cpmg = np.asarray(power_arr)
    tcp = np.asarray(tau_cpmg_arr)

    # For spin 1
    ns_cpmg_2site_expanded.r2eff_ns_cpmg_2site_expanded(
            r20=R20_s1, 
            pA=pA_s1, dw=dw_rad_s1, 
            dw_orig=dw_rad_s1, kex=kex_s1, 
            relax_time=relax_time, 
            inv_relax_time=inv_relax_time, 
            tcp=tcp, 
            back_calc=y_R2_s1, 
            num_cpmg=num_cpmg)
    # For spin 2
    ns_cpmg_2site_expanded.r2eff_ns_cpmg_2site_expanded(
            r20=R20_s2, 
            pA=pA_s2, dw=dw_rad_s2, 
            dw_orig=dw_rad_s2, kex=kex_s2, 
            relax_time=relax_time, 
            inv_relax_time=inv_relax_time, 
            tcp=tcp, 
            back_calc=y_R2_s2, 
            num_cpmg=num_cpmg)
    # Make figure
    f, ax = plt.subplots(1, figsize=(12, 4))
    # Plot
    label_s1 = "sfrq=%.1f MHz\nR20=%.1f rad/s\ndw=%.1f ppm\npA=%.3f \nkex=%.1f rad/s"%(w0_1H_s1, R20_s1, dw_s1, pA_s1, kex_s1)
    plt.plot(x_cpmg_frqs, y_R2_s1, label=label_s1)
    label_s2 = "sfrq=%.1f MHz\nR20=%.1f rad/s\ndw=%.1f ppm\npA=%.3f \nkex=%.1f rad/s"%(w0_1H_s2, R20_s2, dw_s2, pA_s2, kex_s2)
    plt.plot(x_cpmg_frqs, y_R2_s2, label=label_s2)
    # Set labels
    plt.xlabel = "CPMG pulse train frequency v [Hz]"
    plt.ylabel = "R2,eff rad/s"
    ax = plt.gca()
    p_ylim_up = ax.get_ylim()[-1]
    # Round up to nearest 5
    p_ylim_up = p_ylim_up + (- p_ylim_up % 5 )
    ax.set_ylim(0, p_ylim_up)
    ax.set_xlim(0, cpmg_e)
    # Put legend outside
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    # Final call to show.
    plt.show()

# Widgets
from ipywidgets import interact, interactive
import ipywidgets as widgets

widget2 = interactive(ns_cpmg_2site_expanded_calc,
                       relax_time=(0.02, 0.08, 0.01),
                       w0_1H_s1=(500., 1000., 50), w0_1H_s2=(500., 1000., 50),
                       R20_s1=(5.0, 25.0, 1.), R20_s2=(5.0, 25.0, 1.),
                       dw_s1=(0.1, 10., 0.1), dw_s2=(0.1, 10., 0.1),
                       pA_s1=(0.500, 1.000, 0.01), pA_s2=(0.500, 1.000, 0.01),
                       kex_s1=(400., 10000, 200.), kex_s2=(400., 10000, 200.))

#output = widget.children[-1]
#output.layout.height = '500px'
widget2

interactive(children=(FloatSlider(value=0.06, description='relax_time', max=0.08, min=0.02, step=0.01), FloatSlider(value=750.0, description='w0_1H_s1', max=1000.0, min=500.0, step=50.0), FloatSlider(value=750.0, description='w0_1H_s2', max=1000.0, min=500.0, step=50.0), FloatSlider(value=13.9, description='R20_s1', max=25.0, min=5.0, step=1.0), FloatSlider(value=13.9, description='R20_s2', max=25.0, min=5.0, step=1.0), FloatSlider(value=1.02, description='dw_s1', max=10.0, min=0.1), FloatSlider(value=0.69, description='dw_s2', max=10.0, min=0.1), FloatSlider(value=0.87, description='pA_s1', max=1.0, min=0.5, step=0.01), FloatSlider(value=0.5, description='pA_s2', max=1.0, min=0.5, step=0.01), FloatSlider(value=4027.0, description='kex_s1', max=10000.0, min=400.0, step=200.0), FloatSlider(value=4061.0, description='kex_s2', max=10000.0, min=400.0, step=200.0), Output()), _dom_classes=('widget-interact',))

# Code reference in relax

## Target function

In [16]:
#help(relax_disp)
#relax_disp??

In [16]:
#help(relax_disp.Dispersion)

## 'CR72' model
See how the functions are defined

In [17]:
#relax_disp.Dispersion.func_CR72??

In [18]:
#relax_disp.Dispersion.calc_CR72_chi2??

In [19]:
#cr72.r2eff_CR72??

## 'NS CPMG 2-site expanded' model

In [20]:
#relax_disp.Dispersion.func_ns_cpmg_2site_expanded??

In [23]:
#ns_cpmg_2site_expanded.r2eff_ns_cpmg_2site_expanded??